In [1]:
import cv2
import Levenshtein
import optuna
import numpy as np
import pandas as pd
import plotly.express as px
import sklearn.metrics
import ipywidgets
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm.auto import tqdm
tqdm.pandas()

/opt/miniconda/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
%load_ext autoreload
%autoreload 2
from nncomp_molecule import constants

In [3]:
train_df = pd.read_pickle("/work/input/kfujikawa/bms-preprocess-v2/train.pkl").set_index("image_id")
train_df.head()

,InChI,image_path,InChI_length,w,h,w/h,w*h,is_flipped
image_id,,,,,,,,
4435736fd10b,InChI=1S/C65H110O6/c1-4-7-10-13-16-19-22-25-28...,/work/input/bms-molecular-translation/train/4/...,403,1472,788,1.868020,1159936,False
8c0e35ce3f1f,InChI=1S/C60H98O6/c1-4-7-10-13-16-19-22-25-27-...,/work/input/bms-molecular-translation/train/8/...,398,939,309,3.038835,290151,False
934593ad3cae,InChI=1S/C63H106O6/c1-4-7-10-13-16-19-22-25-27...,/work/input/bms-molecular-translation/train/9/...,397,826,661,1.249622,545986,False
c963808e309d,InChI=1S/C59H92O6/c1-4-7-10-13-16-19-22-25-28-...,/work/input/bms-molecular-translation/train/c/...,393,1268,883,1.436014,1119644,False
0aa425d5f5ac,InChI=1S/C62H106O6/c1-4-7-10-13-16-19-22-25-27...,/work/input/bms-molecular-translation/train/0/...,393,918,543,1.690608,498474,False


In [4]:
VALID_CSVs = [
    constants.OUTPUTDIR / "1113_swin_large_bert_384" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "1113_swin_large_bert_384" / "valid_beam=4.csv",
    constants.OUTPUTDIR / "1113_swin_large_bert_384" / "valid_beam=16.csv",
#     constants.OUTPUTDIR / "1113_swin_large_bert_384" / "candidate_0521_1113_swin_large_bert_384.csv",
    constants.OUTPUTDIR / "1106_swin_bert_384" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "1106_swin_bert_384" / "valid_beam=4.csv",
#     constants.OUTPUTDIR / "1106_swin_bert_384" / "candidate_0521_1106_swin_bert_384.csv",
    constants.OUTPUTDIR / "1109_vtnt_bert_512-1024-denoise-5" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "1109_vtnt_bert_512-1024-denoise-5" / "valid_beam=4.csv",
    
    constants.OUTPUTDIR / "9006_1103+1106+1109" / "valid_beam=1.csv",
    constants.OUTPUTDIR / "9006_1103+1106+1109" / "valid_beam=4.csv",
    constants.OUTPUTDIR / "9007_1109+1113" / "valid_beam=4.csv",
#     constants.OUTPUTDIR / "1109_vtnt_bert_512-1024-denoise-5" / "candidate_0521_1109_vtnt_bert_512-1024-denoise-5.csv",
#     constants.OUTPUTDIR / "9004_1102+1105" / "valid_beam=1.csv",
#     constants.OUTPUTDIR / "9005_1102+1105+1106" / "valid_beam=1.csv",
#     constants.OUTPUTDIR / "9006_1103+1106+1109" / "valid_beam=1.csv",
#     constants.OUTPUTDIR / "9006_1103+1106+1109" / "valid_beam=4.csv",
#     constants.OUTPUTDIR / "v30" / "valid_pred.csv",
]
COLUMNS = [
    "image_id",
    "normed_InChI",
    "is_valid",
    "normed_score",
    "InChI_GT",
    "levenshtein",
]
valid_df = pd.concat([
    pd.read_csv(filename, usecols=COLUMNS).assign(model=filename.parent.name)
    for filename in tqdm(VALID_CSVs)
], ignore_index=True)
valid_df = valid_df.drop_duplicates(["image_id", "model", "normed_InChI"])
display(valid_df.groupby("model").agg(**{
    "count": ("image_id", "count"),
    "levenshtein": ("levenshtein", "mean"),
    "is_valid": ("is_valid", "mean"),
    "normed_score": ("normed_score", "mean")
}))

,count,levenshtein,is_valid,normed_score
model,,,,
1106_swin_bert_384,947763,9.939814,0.454798,0.090014
1109_vtnt_bert_512-1024-denoise-5,951360,15.925850,0.495948,0.086438
1113_swin_large_bert_384,3824177,12.774713,0.189188,0.140054
9006_1103+1106+1109,950940,17.048165,0.512284,0.091095
9007_1109+1113,949810,15.080523,0.493025,0.091854


In [6]:
sort_keys = dict(
    image_id=True,
    is_valid=False,
    normed_score=True,
)
valid_ensembled_df = valid_df.sort_values(
    by=list(sort_keys.keys()),
    ascending=list(sort_keys.values()),
).groupby(["image_id", "model"]).first()
valid_ensembled_df.groupby("model").levenshtein.mean()

model
1106_swin_bert_384                   0.928071
1109_vtnt_bert_512-1024-denoise-5    0.922749
1113_swin_large_bert_384             0.828421
9006_1103+1106+1109                  0.786956
9007_1109+1113                       0.859471
Name: levenshtein, dtype: float64

In [7]:
base_score = valid_df.sort_values(
    by=list(sort_keys.keys()),
    ascending=list(sort_keys.values()),
).groupby("image_id").first().levenshtein.mean()
lower_score = valid_df.groupby("image_id").levenshtein.min().mean()

print(f"Levenshtein: {base_score}")
print(f"Levenshtein (lower): {lower_score}")

Levenshtein: 0.7415837867493884
Levenshtein (lower): 0.12317928875211927


In [9]:
valid_ensembled_df = valid_df.groupby(["image_id", "normed_InChI"]).mean()
base_score = valid_ensembled_df.sort_values(
    by=list(sort_keys.keys()),
    ascending=list(sort_keys.values()),
).groupby("image_id").first().levenshtein.mean()
lower_score = valid_df.groupby("image_id").levenshtein.min().mean()

print(f"Levenshtein: {base_score}")
print(f"Levenshtein (lower): {lower_score}")

Levenshtein: 0.7472475342279277
Levenshtein (lower): 0.17425201819989358


In [11]:
MODELS = valid_df.model.unique()

def objective(trial):
    df = valid_df.copy()
    weights = {
        model: trial.suggest_int(model, 0, 10)
        for model in MODELS
    }
    normed_weights = {
        k: v / sum(weights.values())
        for k, v in weights.items()
    }
    df["normed_score"] = df.model.map(normed_weights) * df.normed_score
    df = df.groupby(["image_id", "normed_InChI"]).mean()
    df = df.sort_values(
        by=list(sort_keys.keys()),
        ascending=list(sort_keys.values()),
    ).groupby("image_id").first()
    return df.levenshtein.mean()

study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=100, show_progress_bar=True)
display(study.best_value)
display(study.best_params)

[I 2021-05-21 19:05:13,038] A new study created in memory with name: no-name-1c93b82b-4d37-46b0-ab9b-1dbf4e6b3264


[I 2021-05-21 19:05:26,086] Trial 0 finished with value: 0.7476889187728685 and parameters: {'1113_swin_large_bert_384': 6, '1106_swin_bert_384': 7, '1109_vtnt_bert_512-1024-denoise-5': 6}. Best is trial 0 with value: 0.7476889187728685.
[I 2021-05-21 19:05:38,967] Trial 1 finished with value: 0.7466535213823999 and parameters: {'1113_swin_large_bert_384': 5, '1106_swin_bert_384': 4, '1109_vtnt_bert_512-1024-denoise-5': 7}. Best is trial 1 with value: 0.7466535213823999.
[I 2021-05-21 19:05:51,892] Trial 2 finished with value: 0.7525606491240373 and parameters: {'1113_swin_large_bert_384': 4, '1106_swin_bert_384': 9, '1109_vtnt_bert_512-1024-denoise-5': 10}. Best is trial 1 with value: 0.7466535213823999.
[I 2021-05-21 19:06:04,786] Trial 3 finished with value: 0.7511292431698835 and parameters: {'1113_swin_large_bert_384': 4, '1106_swin_bert_384': 8, '1109_vtnt_bert_512-1024-denoise-5': 5}. Best is trial 1 with value: 0.7466535213823999.
[I 2021-05-21 19:06:17,388] Trial 4 finished wi

0.7393644887570694

{'1113_swin_large_bert_384': 9,
 '1106_swin_bert_384': 1,
 '1109_vtnt_bert_512-1024-denoise-5': 9}